In [23]:
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder,OneHotEncoder
import pickle


In [24]:
data = pd.read_csv("Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [25]:
#drop Irrelevant columns
data= data.drop(["RowNumber","CustomerId","Surname"],axis=1)
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [26]:
#Encode Categorical Columns
label_encoder_gender = LabelEncoder()
data["Gender"] = label_encoder_gender.fit_transform(data["Gender"])
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [27]:
Onehot_encoder_geo = OneHotEncoder()
geo_encoder = Onehot_encoder_geo.fit_transform(data[["Geography"]])

In [28]:
Onehot_encoder_geo.get_feature_names_out(["Geography"])
geo_encoded_df = pd.DataFrame(geo_encoder.toarray(),columns=Onehot_encoder_geo.get_feature_names_out(["Geography"]))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [29]:
data = pd.concat([data.drop(["Geography"],axis=1),geo_encoded_df],axis=1)

In [30]:
##Save Encoders and Scalers
with open("label_encoder_gender.pkl",'wb') as file:
    pickle.dump(label_encoder_gender,file)
with open("Onehot_encoder_geo.pkl",'wb') as file:
    pickle.dump(Onehot_encoder_geo,file)


In [31]:
#Divide Data Set 
X= data.drop(["Exited"], axis=1)
y=data['Exited']

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2, random_state=43)

scalar = StandardScaler()
X_train = scalar.fit_transform(X_train)
X_test = scalar.transform(X_test)

In [32]:
with open("scalar.pkl",'wb') as file:
    pickle.dump(scalar,file)

# ANN Implementation

In [33]:
X_train.shape

(8000, 12)

In [34]:
import tensorflow as tf 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard

model = Sequential([ 
                        Dense(64, activation='relu',input_shape=(X_train.shape[1],)),
                        Dense(32, activation='relu'),
                        Dense(1, activation='sigmoid'),
                    ]
)


c:\Users\mrafi\Documents\ANN\ANN\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [35]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [36]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
loss = tf.keras.losses.BinaryCrossentropy()

In [37]:
model.compile(optimizer=optimizer, loss=loss, metrics=["accuracy"])

In [38]:
##Set up TensorBoard
import datetime
log_dir = "logs/fit"+ datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
#tensorflow_callbacks = TensorBoard(log_dir=log_dir, histogram_freq=1)
import os 
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=os.path.join(log_dir, 'ckpt-{epoch}',".weights.h5"),
    save_weights_only=True,
    save_best_only=False,
    verbose=1
)
tensorflow_callbacks = tf.keras.callbacks.TensorBoard(
    log_dir=log_dir,
    histogram_freq=1,  # Log histograms every epoch
    write_graph=True,  # Log the graph
    update_freq='epoch'  # How often to write logs (batch or epoch)
)

In [39]:
### Set up early stopping 
early_stopping_callbacks = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [40]:
###Train Model 
history = model.fit(X_train, y_train, validation_data=(X_test,y_test), epochs=100, callbacks=[tensorflow_callbacks,early_stopping_callbacks,checkpoint_callback])

Epoch 1/100
231/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8073 - loss: 0.4562
Epoch 1: saving model to logs/fit20240826-171033\ckpt-1\.weights.h5
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8097 - loss: 0.4515 - val_accuracy: 0.8525 - val_loss: 0.3732
Epoch 2/100
229/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8636 - loss: 0.3469
Epoch 2: saving model to logs/fit20240826-171033\ckpt-2\.weights.h5
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8630 - loss: 0.3475 - val_accuracy: 0.8630 - val_loss: 0.3483
Epoch 3/100
227/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8569 - loss: 0.3476
Epoch 3: saving model to logs/fit20240826-171033\ckpt-3\.weights.h5
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8571 - loss: 0.3472 - val_accuracy: 0.8485 - val_loss: 0.3638
Epoch 4/100
235/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8667 - loss: 0.3348
Epoch 4: saving model to logs/fit20240826-171033\ckpt-4\.weights.h5
250/250 ━━━━━━━━━━━━━━━━━━━━ 

In [41]:
model.save("model.h5")

In [42]:
# Load Tensorboard extension 
%reload_ext tensorboard

In [43]:
%tensorboard --logdir logs/fit20240826-123522/

Reusing TensorBoard on port 6010 (pid 17720), started 4:33:11 ago. (Use '!kill 17720' to kill it.)